In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pickle
import time
from bs4 import BeautifulSoup

In [ ]:
def pickle_file(obj, path):
    file = open(path, 'wb')
    pickle.dump(obj, file)
    file.close()

def load_pickle(path):
    file = open(path, 'rb')
    obj = pickle.load(file)
    file.close()
    return obj

### Start Webdriver

In [2]:
driver_path = './WebDriver/chromedriver'
driver = webdriver.Chrome(driver_path)

#   options = ChromeOptions()
#   driver = webdriver.Chrome(driver_path,options=options)

/tmp/ipykernel_20036/2995515328.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


### Load Webpage

In [3]:
driver.get('https://www.nasdaq.com/market-activity/ipos')

#### Navigate to old date

In [42]:
def get_starting_page(driver, starting_date = '01/1995'):
    #Open Date 
    date_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/button'))
        ).click()

    #Set Date Picker Date
    date_picker = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div/input'))
        )
    driver.execute_script(f"arguments[0].value = {starting_date};", date_picker)     
        # .setAttribute('value', '01/1999')

    #Navigate to New Page
    apply_date_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div/button[2]'))
        ).click()

### Build List of Dictionaries From Table

In [62]:
def get_starting_page(driver):
    #Open Date 
    date_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/button'))
        ).click()

    #Set Date Picker Date
    date_picker = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div/input'))
        )
    driver.execute_script("arguments[0].value = '01/1995';", date_picker)     
        # .setAttribute('value', '01/1999')

    #Navigate to New Page
    apply_date_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div/button[2]'))
        ).click()

def generate_data(driver):
    table = driver.find_element(By.XPATH,
        '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[3]/div[5]/div[1]/div[1]/div/table')
    html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    
    url_base = 'https://www.nasdaq.com'
    table_data = []
    # print(soup)
    # if soup.find('th').text == '':
    #     return []

    rows = soup.find_all('tr',{'class':'market-calendar-table__row'})
    for row in rows:
        row_data = {}

        th = row.find('th', {'role':'cell'})
        ticker = th.findChild().text
        if not ticker:
            continue

        row_data[th['data-column']] = ticker
        row_data['link'] = url_base + row.find('a')['href']

        cells = row.find_all('td',{'role':'cell'})        
        for cell in cells:
            row_data[cell['data-column']] = cell.findChild().text
        table_data.append(row_data)
    return table_data

# generate_data(driver)

def get_next_calendar_page(driver):
    next_scroll = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, ".//button[@class='time-belt__next']"))
        ).click()

    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, ".//button[@class='time-belt__item']"))
        )
    if int(next_button.get_attribute('data-year')) <= 2022:
        next_button.click()
    else:
        return False
    return True

# get_next_calendar_page(driver)


[{'proposedTickerSymbol': 'ASDG',
  'link': 'https://www.nasdaq.com/market-activity/ipos/overview?dealId=1416-8081',
  'companyName': 'Super Nova Resources Inc.',
  'filedDate': '07/08/1996',
  'dollarValueOfSharesOffered': '$5,433,750'},
 {'proposedTickerSymbol': 'ADIN',
  'link': 'https://www.nasdaq.com/market-activity/ipos/overview?dealId=676-8350',
  'companyName': 'AVIATION DISTRIBUTORS INC',
  'filedDate': '07/12/1996',
  'dollarValueOfSharesOffered': '$6,000,000'}]

False

In [ ]:
def main_loop(pickup_file = False, starting_date= '01/1995'):
